In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from transformers import default_data_collator
from transformers import BertConfig, BertForSequenceClassification

import evaluate
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split



## Hyperparamètres

In [ ]:
# Tokeniser
max_length = 256 # Longueur maximale des séquences tokenisées

# Capacité du modèle
batch_size = 16 # Taille des batchs
num_epochs = 3 # Nombre d'époques

# Dropout
dropout_global = 0.3 # Dropout global
dropout_attention = 0.4 # Dropout dans les couches d'attention

# Autres paramètres du modèle
#learning_rate = 2e-5 # Taux d'apprentissage
weight_decay = 0.1 # Paramètre de régularisation L2
warmup_steps = 0 # Pas de warmup
load_best_model_at_end = True # Charger le meilleur modèle parmi ceux générés pendant les epochs

# Nombre de labels dans le dataset (ici on est sur de la classification binaire donc 2)
num_labels = 2

In [ ]:
# Détection du matériel à disposition pour l'entrainement du modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cpu = device.type == "cpu"

if str(device) == "cuda":
    device_name = torch.cuda.get_device_name()
else:
    device_name = "CPU nul..."

print("Device:", device_name)

## Import des données

In [ ]:
# Charger les données
train_df = pd.read_csv("../preprocessing_with_BLIP/train_blip.csv")
test_df = pd.read_csv("../preprocessing_with_BLIP/test_blip.csv")

# Nettoyage des données
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# Compter le nombre d'exemples dans chaque classe dans le dataset d'entraînement
train_label_counts = train_df["label"].value_counts()
print("Répartition des labels dans le dataset d'entraînement :\n", train_label_counts)
print()

# Compter le nombre d'exemples dans chaque classe dans le dataset de test
test_label_counts = test_df["label"].value_counts()
print("Répartition des labels dans le dataset de test :\n", test_label_counts)


In [ ]:
# Tokenizer de DistilBERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
class MemeDataset(Dataset):
    def __init__(self, texts, descs, labels):
        self.texts = texts
        self.descs = descs
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        desc = self.descs[idx]
        label = self.labels[idx]

        encoding = tokenizer(desc, text, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

## Préparation et création du modèle

In [ ]:
from transformers import DistilBertConfig

# Séparer 10% des données d'entraînement pour la validation
train_texts, val_texts, train_descs, val_descs, train_labels, val_labels = train_test_split(
    train_df["text"], train_df["img_desc"], train_df["label"], test_size=0.1, random_state=42
)

# Créer les datasets pour Trainer
train_dataset = MemeDataset(train_texts.tolist(), train_descs.tolist(), train_labels.tolist())
val_dataset = MemeDataset(val_texts.tolist(), val_descs.tolist(), val_labels.tolist())
test_dataset = MemeDataset(test_df["text"].tolist(), test_df["img_desc"].tolist(), test_df["label"].tolist())

# Charger la configuration du dropout
config = BertConfig.from_pretrained("bert-base-uncased")
config.dropout = dropout_global  # Appliquer du dropout globalement
config.attention_dropout = dropout_attention  # Appliquer du dropout dans les couches d'attention
config.num_labels=num_labels

# Charger le modèle BERT (sur le device détecté)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)
model.to(device)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Configuration de l'entraînement optimisée pour CPU
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=num_epochs,  # Réduire à 1 époque pour éviter un entraînement trop long
    per_device_train_batch_size=batch_size,  # Réduire pour éviter saturation mémoire
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    lr_scheduler_type="cosine",
    logging_dir="./logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    use_cpu=use_cpu,
    load_best_model_at_end = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator  # Ajout de cette ligne
)

## Entrainement du modèle

In [ ]:
# Entraînement du modèle (cela prendra du temps sur CPU)
print(f"Modèle chargé sur {device_name}")
print("Début de l'entrainement...")
trainer.train()

In [ ]:
# Obtenir les prédictions sur le train set
predictions = trainer.predict(train_dataset)
logits = predictions.predictions
y_train = np.argmax(logits, axis=-1)

# Obtenir les prédictions sur le test set
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
y_pred = np.argmax(logits, axis=-1)

# Calculer la distribution des prédictions pour le train et le test
unique_train, counts_train = np.unique(y_train, return_counts=True)
train_counts = dict(zip(unique_train, counts_train))

unique_test, counts_test = np.unique(y_pred, return_counts=True)
test_counts = dict(zip(unique_test, counts_test))



# Charger la métrique d'accuracy
accuracy_metric = evaluate.load("accuracy")

# Obtenir les prédictions sur le train et dev set
train_predictions = trainer.predict(train_dataset)
train_logits = train_predictions.predictions
train_labels = train_predictions.label_ids
train_preds = np.argmax(train_logits, axis=-1)

test_predictions = trainer.predict(test_dataset)
test_logits = test_predictions.predictions
test_labels = test_predictions.label_ids
test_preds = np.argmax(test_logits, axis=-1)

# Calculer l'accuracy
train_acc = accuracy_metric.compute(predictions=train_preds, references=train_labels)['accuracy']
test_acc = accuracy_metric.compute(predictions=test_preds, references=test_labels)['accuracy']

# Calculer le F1 score
train_f1 = f1_score(train_labels, train_preds, average='binary')
test_f1 = f1_score(test_labels, test_preds, average='binary')

# Distribution des prédictions en pourcentage
def distribution(y_pred):
    unique, counts = np.unique(y_pred, return_counts=True)
    total = len(y_pred)
    distribution = {label: round((count / total) * 100, 2) for label, count in zip(unique, counts)}
    return distribution

train_dist = distribution(train_preds)
dev_dist = distribution(test_preds)

In [ ]:
# Afficher la distribution avec les pourcentages
print("=== Distribution des prédictions sur TRAIN ===")
total_train = sum(counts_train)
for label, count in train_counts.items():
    percentage = (count / total_train) * 100
    print(f"Classe {label}: {count} ({percentage:.2f}%)")
print("=================================")

print("=== Distribution des prédictions sur TEST ===")
total_test = sum(counts_test)
for label, count in test_counts.items():
    percentage = (count / total_test) * 100
    print(f"Classe {label}: {count} ({percentage:.2f}%)")
print("=================================")


# Affichage des résultats
print("===== Train Accuracy =====")
acc = round(train_acc * 100, 2)
print(f"Accuracy: {acc}%")

acc = round(train_f1 * 100, 2)
print(f"F1 Score: {acc}%")

print("\n===== Dev Accuracy =====")
acc = round(test_acc * 100, 2)
print(f"Accuracy: {acc}%")

acc = round(test_f1 * 100, 2)
print(f"F1 Score: {acc}%")

## Export des résultats et hyperparamètres associés

In [ ]:
import csv
import os
import numpy as np

# Fonction pour formater les dictionnaires contenant des np.int64
def format_dict(d):
    return " , ".join(f"{int(k)} : {int(v)}" for k, v in d.items())

# Hyperparamètres et résultats
params = {
    "max_length": max_length,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "dropout_global": dropout_global,
    "dropout_attention": dropout_attention,
    "weight_decay": weight_decay,
    "warmup_steps": warmup_steps,
    "load_best_model_at_end": load_best_model_at_end,
    "num_labels": num_labels,
    "train_acc": round(train_acc, 4),
    "train_f1_score": round(train_f1, 4),
    "test_acc": round(test_acc, 4),
    "test_f1_score": round(test_f1, 4),
    "train_pred_distribution": format_dict(train_counts),
    "test_pred_distribution": format_dict(test_counts)
}

# Nom du fichier CSV
output_file = "hyperparameters_results.csv"

# Vérifier si le fichier existe déjà
if os.path.exists(output_file):
    # Lire le fichier existant
    with open(output_file, mode='r', newline='', encoding='utf-8-sig') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Déterminer le numéro de la prochaine colonne (valeurs_X)
    header = rows[0]
    existing_value_columns = [col for col in header[1:] if col.startswith("valeurs_")]
    if existing_value_columns:
        last_index = max(int(col.split("_")[1]) for col in existing_value_columns)
        new_col_name = f"valeurs_{last_index + 1}"
    else:
        new_col_name = "valeurs_1"

    # Ajouter le nom de la nouvelle colonne dans l'en-tête
    header.append(new_col_name)

    # Mettre à jour les lignes avec les nouvelles valeurs
    for i in range(1, len(rows)):  # On commence à 1 pour ignorer l'en-tête
        key = rows[i][0]
        if key in params:
            rows[i].append(params[key])  # Ajouter la valeur correspondante
        else:
            rows[i].append("")  # Ajouter une cellule vide pour les lignes sans correspondance

    # Écrire les nouvelles données dans le fichier
    with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

else:
    # Fichier n'existe pas encore, le créer et écrire les données
    with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)

        # Écrire l'en-tête
        writer.writerow(["Hyperparamètres et Résultats", "valeurs_1"])

        # Écrire les valeurs
        for key, value in params.items():
            writer.writerow([key, value])

print(f"Données exportées avec succès dans {output_file}!")
